In [1]:
from visuals import *
from glob import glob
from datetime import datetime
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

In [20]:
# Determine selling cap rate from remaining rent
def scapfromrent(dates, emonth, rate, fyrrent, rentinc, incperiod, expmonth):
    rent = rentcf(fyrrent, rentinc, incperiod, dates, expmonth)
    remdates = dates[emonth:]
    print(remdates[-1])
    remrent = rent[emonth:]
    t0 = remdates[0]
    diff = (remdates - t0).days.values/360.
    Pf = xnpv(rate, remrent, diff)
    print(Pf)
    return remrent[0]*12./Pf

In [3]:
# Load LIBOR
curves = glob('*.xlsx')
# temp = pd.read_excel(curves[0]).values
# libor = temp.T[18]

In [4]:
temp2 = pd.read_excel(curves[1]).values[2:]
libor = temp2.T[2]/100.

In [5]:
nprop = 11
names = ['200 Aurora Way', '9950 Wakeman Drive', '1900 SW 44th Ave.', '100 Center Ave N', '215 Mill Street SW', '1090 Pacific Ave', '1600 2nd Street NE',
         '3000 Tremont Road', '8700 W Doe Ave', '10401 Bunsen Way', '660 National Turnpike']
sdate = np.datetime64('2018-10-01') # acquisition date
# sdate = datetime(2018, 10, 1)
bcaps = np.ones(nprop)*0.09 # buying cap rates same for all properties
expdates = pd.DatetimeIndex(['3/31/2043', '3/31/2043', '12/31/2033', '8/31/2036', '8/31/2036', '8/23/2033', '8/31/2036', '8/31/2033', '8/31/2033', '3/31/2035',
                              '3/31/2035'])

In [6]:
emonths = [3]
for i in range(1, 11):
    emonths.append(emonths[i-1] +2) # sell properties every other month starting month 3

In [7]:
fyrrents = [1475667., 528246., 1882931., 65446., 112446., 526069., 224876., 1476287., 446033., 684789., 940794.]
rentincs = np.array([1.5, 1.5, 2., 2., 2., 2., 2., 2., 2., 1.5, 1.5])/100.
incperiods = np.ones(11)

In [22]:
scaps = np.zeros(11)
d = pd.date_range(sdate, '3/31/2043', freq = 'M')
scaps[0] = scapfromrent(d, emonths[0], 0.1, fyrrents[0], rentincs[0], 1, d[-1].month)
scaps[1] = scapfromrent(d, emonths[1], 0.1, fyrrents[1], rentincs[1], 1, d[-1].month)

2043-03-31 00:00:00
15682185.637521857
2043-03-31 00:00:00
5613048.039582706


In [23]:
scaps

array([0.0940983 , 0.09411037, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [9]:
scaps[0] = 0.093064645
scaps[1] = 0.09308034

In [10]:
scaps[2:] = 0.085

In [11]:
ltvs = np.ones(11)*0.55
r0 = 0.025
acosts = np.ones(11)/100.
ecosts = np.ones(11)*4/100.
fyroexps = np.zeros(nprop)
oexpincs = np.zeros(nprop)
aperiod = np.inf
lfee = 0.012
aintrate = 0.
repprem = 1.2

In [12]:
alldates, amort, propcf, interest, loanpay, debt = portcf(names, sdate, bcaps, scaps, emonths, fyrrents, rentincs, incperiods, ltvs, lfee, r0, libor, acosts, ecosts, fyroexps, oexpincs, aperiod, aintrate, repprem, expdates)

200 Aurora Way,,,,
Month #,0,1,2,3
Dates,2018-10-31,2018-11-30,2018-12-31,2019-01-31
Property buy/sell,"-$16,560,263.00",$0.00,$0.00,"$15,222,110.61"
Rent,$0.00,"$122,972.25","$122,972.25",$0.00
Operating expenses,-$0.00,-$0.00,-$0.00,-$0.00
Total cash flow,"-$16,560,263.00","$122,972.25","$122,972.25","$15,222,110.61"


9950 Wakeman Drive,,,,,,
Month #,0,1,2,3,4,5
Dates,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31
Property buy/sell,"-$5,928,094.00",$0.00,$0.00,$0.00,$0.00,"$5,448,155.43"
Rent,$0.00,"$44,020.50","$44,020.50","$44,020.50","$44,020.50",$0.00
Operating expenses,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00
Total cash flow,"-$5,928,094.00","$44,020.50","$44,020.50","$44,020.50","$44,020.50","$5,448,155.43"


1900 SW 44th Ave.,,,,,,,,
Month #,0,1,2,3,4,5,6,7
Dates,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31
Property buy/sell,"-$21,130,670.11",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$21,691,365.12"
Rent,$0.00,"$156,910.92","$156,910.92","$160,049.13","$160,049.13","$160,049.13","$160,049.13",$0.00
Operating expenses,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00
Total cash flow,"-$21,130,670.11","$156,910.92","$156,910.92","$160,049.13","$160,049.13","$160,049.13","$160,049.13","$21,691,365.12"


100 Center Ave N,,,,,,,,,,
Month #,0,1,2,3,4,5,6,7,8,9
Dates,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31
Property buy/sell,"-$734,449.56",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$739,154.82"
Rent,$0.00,"$5,453.83","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$5,453.83",$0.00
Operating expenses,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00
Total cash flow,"-$734,449.56","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$739,154.82"


215 Mill Street SW,,,,,,,,,,,,
Month #,0,1,2,3,4,5,6,7,8,9,10,11
Dates,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30
Property buy/sell,"-$1,261,894.00",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$1,295,377.92"
Rent,$0.00,"$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50",$0.00
Operating expenses,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00
Total cash flow,"-$1,261,894.00","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$1,295,377.92"


...

1600 2nd Street NE,,,,,,,,,,,,,,,,
Month #,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Dates,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31
Property buy/sell,"-$2,523,608.44",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$2,590,571.52"
Rent,$0.00,"$18,739.67","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$19,114.46","$19,114.46","$19,114.46","$19,114.46",$0.00
Operating expenses,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00
Total cash flow,"-$2,523,608.44","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$18,739.67","$19,114.46","$19,114.46","$19,114.46","$19,114.46","$2,590,571.52"


3000 Tremont Road,,,,,,,,,,,,,,,,,,
Month #,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Dates,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31
Property buy/sell,"-$16,567,220.78",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$17,006,826.24"
Rent,$0.00,"$123,023.92","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$125,484.40","$125,484.40","$125,484.40","$125,484.40","$125,484.40","$125,484.40",$0.00
Operating expenses,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00
Total cash flow,"-$16,567,220.78","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$123,023.92","$125,484.40","$125,484.40","$125,484.40","$125,484.40","$125,484.40","$125,484.40","$17,006,826.24"


8700 W Doe Ave,,,,,,,,,,,,,,,,,,,,
Month #,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Dates,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31
Property buy/sell,"-$5,005,481.44",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$5,138,300.16"
Rent,$0.00,"$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,912.80","$37,912.80","$37,912.80","$37,912.80","$37,912.80","$37,912.80","$37,912.80","$37,912.80",$0.00
Operating expenses,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00
Total cash flow,"-$5,005,481.44","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,169.42","$37,912.80","$37,912.80","$37,912.80","$37,912.80","$37,912.80","$37,912.80","$37,912.80","$37,912.80","$5,138,300.16"


10401 Bunsen Way,,,,,,,,,,,,,,,,,,,,,,
Month #,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Dates,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31
Property buy/sell,"-$7,684,854.33",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$7,967,850.32"
Rent,$0.00,"$57,065.75","$57,065.75","$57,065.75","$57,065.75","$57,065.75","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$58,790.56","$58,790.56","$58,790.56",$0.00
Operating expenses,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00
Total cash flow,"-$7,684,854.33","$57,065.75","$57,065.75","$57,065.75","$57,065.75","$57,065.75","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$57,921.74","$58,790.56","$58,790.56","$58,790.56","$7,967,850.32"


660 National Turnpike,,,,,,,,,,,,,,,,,,,,,,,,
Month #,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
Dates,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30
Property buy/sell,"-$10,557,799.33",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$10,946,591.98"
Rent,$0.00,"$78,399.50","$78,399.50","$78,399.50","$78,399.50","$78,399.50","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$80,769.12","$80,769.12","$80,769.12","$80,769.12","$80,769.12",$0.00
Operating expenses,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00,-$0.00
Total cash flow,"-$10,557,799.33","$78,399.50","$78,399.50","$78,399.50","$78,399.50","$78,399.50","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$79,575.49","$80,769.12","$80,769.12","$80,769.12","$80,769.12","$80,769.12","$10,946,591.98"


In [13]:
df = viewportcf(names, alldates, amort, propcf, interest, loanpay, debt)

In [14]:
viewret(*returns(np.array(amort) + np.sum(propcf, axis = 0) + np.array(interest) +np.array(loanpay), alldates))

Returns,Value
Inflow,"$48,613,525.27"
Outflow,"$43,360,536.60"
Profit,"$5,252,988.66"
MOIC,1.121
IRR,10.172%


In [15]:
df.to_excel('results.xlsx')

In [16]:
df

,,,,,,,,,,,,,,,,,,,,,,,,
Month #,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
Dates,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30
Starting debt,$0.00,"$51,110,791.11","$51,110,791.11","$51,110,791.11","$40,289,233.11","$40,289,233.11","$36,415,429.11","$36,415,429.11","$22,607,268.44","$22,607,268.44","$22,127,331.11","$22,127,331.11","$21,302,727.11","$21,302,727.11","$17,444,887.78","$17,444,887.78","$15,795,797.11","$15,795,797.11","$4,969,692.44","$4,969,692.44","$1,698,783.78","$1,698,783.78",$0.00,$0.00
Amortization,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
200 Aurora Way,"-$16,560,263.00","$122,972.25","$122,972.25","$15,222,110.61",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
9950 Wakeman Drive,"-$5,928,094.00","$44,020.50","$44,020.50","$44,020.50","$44,020.50","$5,448,155.43",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
1900 SW 44th Ave.,"-$21,130,670.11","$156,910.92","$156,910.92","$160,049.13","$160,049.13","$160,049.13","$160,049.13","$21,691,365.12",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
100 Center Ave N,"-$734,449.56","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$5,453.83","$739,154.82",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
215 Mill Street SW,"-$1,261,894.00","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$9,370.50","$1,295,377.92",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
1090 Pacific Ave,"-$5,903,663.22","$43,839.08","$43,839.08","$43,839.08","$43,839.08","$43,839.08","$43,839.08","$43,839.08","$43,839.08","$43,839.08","$43,839.08","$44,715.87","$44,715.87","$6,060,314.88",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00


In [17]:
floatdf = convtofloat(df.values[2:])
viewret(*returns(floatdf[-1], df.values[1]))

Returns,Value
Inflow,"$48,613,525.27"
Outflow,"$43,360,536.60"
Profit,"$5,252,988.67"
MOIC,1.121
IRR,10.172%
